## **Dueling DQN**

Extends DQN by splitting the Q-value into state-value and advantage-value components for better learning.

**Imports**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np
from collections import deque
import random

**Define the Dueling DQN architecture**

In [ ]:
class DuelingDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DuelingDQN, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.value = nn.Sequential(
            nn.Linear(hidden_size, 1)
        )
        self.advantage = nn.Sequential(
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = self.feature(x)
        value = self.value(x)
        advantage = self.advantage(x)
        return value + advantage - advantage.mean()

**Initialize environment and parameters**

In [ ]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
hidden_size = 128
output_size = env.action_space.n
model = DuelingDQN(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
epsilon_decay = 0.995
epsilon_min = 0.01
batch_size = 64
memory = deque(maxlen=10000)
target_update_frequency = 10
steps_done = 0

# Function to select action
def select_action(state):
    global epsilon
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32)
            q_values = model(state_tensor)
            return torch.argmax(q_values).item()

**Function to optimize the model**

In [ ]:
def optimize_model():
    if len(memory) < batch_size:
        return
    transitions = random.sample(memory, batch_size)
    batch = list(zip(*transitions))
    states, actions, rewards, next_states, dones = batch
    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.long)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    q_values = model(states)
    next_q_values = model(next_states)
    next_q_value = next_q_values.max(1)[0]
    expected_q_values = rewards + (gamma * next_q_value * (1 - dones))

    loss = loss_fn(q_values.gather(1, actions.unsqueeze(1)).squeeze(1), expected_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

**Training loop**

In [ ]:
for episode in range(1000):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = select_action(state)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        memory.append((state, action, reward, next_state, done))
        state = next_state
        optimize_model()
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    if episode % target_update_frequency == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")